# Introduction to Tensorflow
In this notebook, we are learning some basic functions of Tensorflow and making a linear model

In [10]:
#Tensorflow package
import tensorflow as tf

# Building Computation Graph
A computational graph is a series of TensorFlow operations arranged into a graph of nodes. Let's build a simple computational graph. Each node takes zero or more tensors as inputs and produces a tensor as an output. One type of node is a constant. Like all TensorFlow constants, it takes no inputs, and it outputs a value it stores internally. We can create two floating point Tensors node1 and node2

In [12]:
#Creating two nodes with constant value
node1 = tf.constant(3.0) # also tf.float32 implicitly
node2 = tf.constant(4.0) 
print(node1, node2)

Tensor("Const_6:0", shape=(), dtype=float32) Tensor("Const_7:0", shape=(), dtype=float32)


# Starting a session :
A session encapsulates the control and state of the TensorFlow runtime.

In [13]:
#To evaluate the nodes we use Session
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [14]:
#Adding two nodes into a third node
node3 = tf.add(node1, node2)
print("node3: ", node3)
print("sess.run(node3): ", sess.run(node3))

node3:  Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3):  7.0


# Using Placeholders :
As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.
The preceding three lines are a bit like a function or a lambda in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument to the run method to feed concrete values to the placeholders:

In [18]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # + provides a shortcut for tf.add(a, b)

print(sess.run(adder_node, {a:4, b:5}))
print(sess.run(adder_node, {a:[1, 7], b:[2, 4]}))

9.0
[  3.  11.]


In [16]:
#We can make the computational graph more complex by adding another operation. 
#For example:
add_and_triple = adder_node * 3
print(sess.run(add_and_triple, {a:4, b:5}))

27.0


# Making a Linear Model
Explanation as well as the coding part

## Using Variables :
In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [54]:
#Creating a linear model
w = tf.Variable([.3], dtype=tf.float32) #Important to write dtype or operator 
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = w * x + b

### How to initialise Variables :
Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [55]:
init = tf.global_variables_initializer()
sess.run(init)

In [56]:
print(sess.run(linear_model, {x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


### What is a LOSS Function?
A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum:

In [57]:
#Evaluating the data on training data
y = tf.placeholder(tf.float32)
square_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(square_deltas)
print(sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

23.66


### Assigning values to Variables:
We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign. For example, W=-1 and b=1 are the optimal parameters for our model. We can change W and b accordingly:

In [58]:
fixW = tf.assign(w, [-1])
fixB = tf.assign(b, [1])
sess.run([fixW, fixB])
print(sess.run(loss, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]}))

0.0


### We guessed the "perfect" values of W and b, but the whole point of machine learning is to find the correct model parameters automatically. We will show how to accomplish this in the next section.

# Making a Gradient Descent:
However, TensorFlow provides optimizers that slowly change each variable in order to minimize the loss function. The simplest optimizer is gradient descent. It modifies each variable according to the magnitude of the derivative of loss with respect to that variable. In general, computing symbolic derivatives manually is tedious and error-prone. Consequently, TensorFlow can automatically produce derivatives given only a description of the model using the function tf.gradients.

In [59]:
optimiser = tf.train.GradientDescentOptimizer(0.01)
train = optimiser.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
    sess.run(train, {x:[1, 2, 3, 4], y:[0, -1, -2, -3]})

print(sess.run([w, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# Complete Code down below:

In [63]:
#Package for tensorflow
import tensorflow as tf

#Model parameters
w = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

#Input and Output
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

#Making a linear model
linear_model = w * x + b

#Loss
loss = tf.reduce_sum(tf.square(linear_model - y))

#Optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

#train
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

#Initializing
init = tf.global_variables_initializer()

#Starting the session
sess = tf.Session()
sess.run(init)

#Training loop
for i in range(1000):
    sess.run(train, {x:x_train, y:y_train})
    
# evaluate training accuracy
curr_w, curr_b, curr_loss = sess.run([w, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_w, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11
